In [1]:
import torch

def noncon_vector_function(input_data, centers, amplitudes, sigmas, transform_matrix=None, center_batch_size=200):
    """
    Compute the vector field at the given input points using Gaussian contributions, with optional linear transformation.
    
    Args:
        input_data (torch.Tensor): Tensor of shape (num_samples, dimension) containing the input points.
        centers (torch.Tensor): Tensor of shape (num_centers, dimension) containing the Gaussian centers.
        amplitudes (torch.Tensor): Tensor of shape (num_centers,) containing the Gaussian amplitudes.
        sigmas (torch.Tensor): Tensor of shape (num_centers,) containing the Gaussian sigmas.
        transform_matrix (torch.Tensor): Tensor of shape (dimension, dimension) describing how to mix the output components. 
                                        If None, no transformation is applied.
        center_batch_size (int): Maximum number of centers to process at a time. Default is 200.
    
    Returns:
        force (torch.Tensor): Tensor of shape (num_samples, dimension) containing the transformed vector field values.
    """
    num_samples = input_data.shape[0]
    dimension = input_data.shape[1]
    num_centers = centers.shape[0]

    # Initialize the output tensor to store the force values
    force = torch.zeros((num_samples, dimension), device=input_data.device)

    # Process centers, amplitudes, and sigmas in batches
    for i in range(0, num_centers, center_batch_size):
        # Get the current batch of centers, amplitudes, and sigmas
        batch_end = min(i + center_batch_size, num_centers)
        batch_centers = centers[i:batch_end]  # Shape: (batch_size, dimension)
        batch_amplitudes = amplitudes[i:batch_end]  # Shape: (batch_size,)
        batch_sigmas = sigmas[i:batch_end]  # Shape: (batch_size,)

        # Expand input_data and batch_centers for broadcasting
        input_data_expanded = input_data.unsqueeze(1)  # Shape: (num_samples, 1, dimension)
        centers_expanded = batch_centers.unsqueeze(0)  # Shape: (1, batch_size, dimension)

        # Compute differences between input points and batch centers
        diff = input_data_expanded - centers_expanded  # Shape: (num_samples, batch_size, dimension)

        # Compute the squared distances and Gaussian exponent terms
        squared_distances = torch.sum(diff**2, dim=2)  # Shape: (num_samples, batch_size)
        exp_terms = torch.exp(-squared_distances / (2 * batch_sigmas**2))  # Shape: (num_samples, batch_size)

        # Compute the gradients for each dimension
        grad = (batch_amplitudes / batch_sigmas**2).unsqueeze(0).unsqueeze(-1) * diff * exp_terms.unsqueeze(-1)  # Shape: (num_samples, batch_size, dimension)

        # Sum contributions from the current batch of centers
        batch_force = -torch.sum(grad, dim=1)  # Shape: (num_samples, dimension)

        # Accumulate the batch results in the output tensor
        force += batch_force

    # Apply transformation if specified
    if transform_matrix is not None:
        force = torch.einsum('ij,sj->si', transform_matrix, force)  # Shape: (num_samples, dimension)

    return force

In [4]:
# Test parameters
dimension = 3
num_samples = 10
num_centers = 100

# Generate random input data, centers, amplitudes, and sigmas
input_data = torch.randn((num_samples, dimension))
centers = torch.randn((num_centers, dimension))
amplitudes = torch.randn((num_centers,))
sigmas = torch.abs(torch.randn((num_centers,))) + 0.1  # Ensure sigmas > 0

# Generate a random transformation matrix (mostly zeros, with some swaps/scales)
transform_matrix = torch.zeros((dimension, dimension))
for i in range(dimension):
    # Randomly choose another component to swap with or scale
    j = torch.randint(0, dimension, (1,)).item()
    transform_matrix[i, j] = torch.randn(1).item() * 2  - 1 # Random scaling

print("Transformation matrix:")
print(transform_matrix)

# Compute original and transformed forces
original_force = noncon_vector_function(input_data, centers, amplitudes, sigmas)
transformed_force = noncon_vector_function(input_data, centers, amplitudes, sigmas, transform_matrix)

# Print some samples to verify transformation
print("\nOriginal force (sample 0):", original_force[0])
print("Transformed force (sample 0):", transformed_force[0])

# Check if the transformed field is non-conservative (Jacobian not symmetric)
sample_point = input_data[0].unsqueeze(0)  # Pick a single point to test
sample_point.requires_grad = True

# Compute Jacobian of the transformed force
force_at_point = noncon_vector_function(sample_point, centers, amplitudes, sigmas)
jacobian = torch.zeros((dimension, dimension))
for i in range(dimension):
    grad_output = torch.zeros_like(force_at_point)
    grad_output[0, i] = 1.0
    gradients = torch.autograd.grad(force_at_point, sample_point, grad_outputs=grad_output, create_graph=True)[0]
    jacobian[i] = gradients[0]

print("\nJacobian of original force at sample 0:")
print(jacobian)

# Check symmetry (if not symmetric, field is non-conservative)
is_symmetric = torch.allclose(jacobian, jacobian.t(), atol=1e-6)
print("\nIs the original Jacobian symmetric (conservative)?", is_symmetric)

# Compute Jacobian of the transformed force
force_at_point = noncon_vector_function(sample_point, centers, amplitudes, sigmas, transform_matrix)
jacobian = torch.zeros((dimension, dimension))
for i in range(dimension):
    grad_output = torch.zeros_like(force_at_point)
    grad_output[0, i] = 1.0
    gradients = torch.autograd.grad(force_at_point, sample_point, grad_outputs=grad_output, create_graph=True)[0]
    jacobian[i] = gradients[0]

print("\nJacobian of transformed force at sample 0:")
print(jacobian)

# Check symmetry (if not symmetric, field is non-conservative)
is_symmetric = torch.allclose(jacobian, jacobian.t(), atol=1e-6)
print("\nIs the transformed Jacobian symmetric (conservative)?", is_symmetric)

Transformation matrix:
tensor([[-1.3150,  0.0000,  0.0000],
        [ 0.0000,  0.0000, -0.2016],
        [ 0.0000, -0.9597,  0.0000]])

Original force (sample 0): tensor([-2.9441,  1.9333, -0.0315])
Transformed force (sample 0): tensor([ 3.8715,  0.0064, -1.8554])

Jacobian of original force at sample 0:
tensor([[ 2.5398, -2.5748, -0.8971],
        [-2.5748, -3.4490,  0.5242],
        [-0.8971,  0.5242, -3.2213]], grad_fn=<CopySlices>)

Is the original Jacobian symmetric (conservative)? True

Jacobian of transformed force at sample 0:
tensor([[-3.3399,  3.3859,  1.1797],
        [ 0.1809, -0.1057,  0.6495],
        [ 2.4711,  3.3100, -0.5030]], grad_fn=<CopySlices>)

Is the transformed Jacobian symmetric (conservative)? False
